In [17]:
import os
import shutil
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import librosa.display
import gc
import numpy as np
import urllib

In [18]:
dataset_dir_p = '../dataset/STFT/presences/' # Folder where dataset will be stored (presences)
dataset_dir_a = '../dataset/STFT/absences/' # Folder where dataset will be stored (absences)
recording_dir = '../recordings/' # Folder holding recordings

#### csv from Arbimon ###
path = '../sound_annotation_files/'
csv = sorted(os.listdir(path))
sound_annotation_files = []
for i in csv:
    if i.endswith('.csv'):
        sound_annotation_files.append(path+i)
#########################

sampling_rate = 44100 

In [21]:
if len(sound_annotation_files)==1:
    labels= pd.read_csv(sound_annotation_files[0])
elif len(sound_annotation_files)==0:
    print('Must provide an annotation file')
elif len(sound_annotation_files)>1:
    labels = pd.read_csv(sound_annotation_files[0])
    for i in sound_annotation_files[1:]:
        tmp = pd.read_csv(i)
        labels = pd.concat([labels,tmp])
labels.head()

,id,recording,site,year,month,day,hour,minute,species,songtype,x1,x2,y1,y2,frequency,validated,url,score,site_id
0,168185976,MON_20200331_180000.p5.flac,Angachilla,2020,3,31,18,4,Agelasticus thilius,Alternative Song,40.095057,41.528889,1205.859375,7235.15625,22050,(not validated),https://arbimon.rfcx.org/api/project/urban-wet...,0.218565,22306
1,168185910,MON_20200331_120000.p4.flac,Angachilla,2020,3,31,12,3,Agelasticus thilius,Alternative Song,11.662222,13.096054,1205.859375,7235.15625,22050,(not validated),https://arbimon.rfcx.org/api/project/urban-wet...,0.193405,22306
2,168185935,MON_20200331_150000.p2.flac,Angachilla,2020,3,31,15,1,Agelasticus thilius,Alternative Song,40.692971,42.126803,1205.859375,7235.15625,22050,(not validated),https://arbimon.rfcx.org/api/project/urban-wet...,0.193366,22306
3,168185786,MON_20200331_010000.p4.flac,Angachilla,2020,3,31,1,3,Agelasticus thilius,Alternative Song,47.316463,48.750295,1205.859375,7235.15625,22050,(not validated),https://arbimon.rfcx.org/api/project/urban-wet...,0.188449,22306
4,168185887,MON_20200331_110000.p1.flac,Angachilla,2020,3,31,11,0,Agelasticus thilius,Alternative Song,43.084626,44.518458,1205.859375,7235.15625,22050,(not validated),https://arbimon.rfcx.org/api/project/urban-wet...,0.185002,22306


In [6]:
'''
callshape = []
for i in range(len(labels)):
    callshape.append(labels.iloc[i].species.replace(' ', '_') + '-' + labels.iloc[i].songtype.replace(' ', '_'))
'''

In [8]:
#labels.insert(loc=10, column='callshape', value=callshape)

In [22]:
#labels

In [23]:
presences = labels[labels.validated == 'present']
absences = labels[labels.validated == 'not present']
not_validated = labels[labels.validated == '(not validated)']
print(len(presences), len(absences), len(not_validated))

21685 114646 4132764


In [24]:
'''
not_validated.to_csv('../sound_annotation_files_splited/not_validated/not_validated.csv') #Data to augmentation
presences.to_csv('../sound_annotation_files_splited/presences/presences.csv') 
absences.to_csv('../sound_annotation_files_splited/absences/absences.csv') 
'''

In [25]:
print('Number of presences for each specie\n')

for i in sorted(list(set(presences.species))):
    print(str(i)+'\t\t'+str(len(presences[presences.species==i])))

Number of presences for each specie

Agelasticus thilius		5014
Anairetes parulus		302
Cistothorus platensis		1434
Colorhamphus parvirostris		1256
Curaeus curaeus		603
Elaenia albiceps		4137
Eugralla paradoxa		459
Gallinago paraguaiae		1010
Glaucidium nana		160
Mareca sibilatrix		1052
Pardirallus sanguinolentus		1318
Phleocryptes melanops		639
Porzana spiloptera		1413
Scytalopus magellanicus		1018
Sicalis luteola		1160
Troglodytes aedon		710


In [26]:
print('Number of absences for each specie\n')

for i in sorted(list(set(absences.species))):
    print(str(i)+'\t\t'+str(len(absences[absences.species==i])))

Number of absences for each specie

Agelasticus thilius		9540
Anairetes parulus		1061
Cistothorus platensis		8515
Colorhamphus parvirostris		17411
Curaeus curaeus		1003
Elaenia albiceps		19549
Eugralla paradoxa		904
Gallinago paraguaiae		14993
Glaucidium nana		11202
Mareca sibilatrix		1728
Pardirallus sanguinolentus		12146
Phleocryptes melanops		1078
Porzana spiloptera		12231
Scytalopus magellanicus		1117
Sicalis luteola		1168
Troglodytes aedon		1000


In [13]:
# Download audio from Arbimon:

### presences ###
k = 0
for i in list(set(presences.url)): # loop over recordings
    tmp = presences[presences.url==i]
    audio_filename = tmp.iloc[0].url.split('download/')[1]
    if not os.path.exists(recording_dir+audio_filename):
        try:
            urllib.request.urlretrieve(tmp.iloc[0].url, recording_dir+audio_filename)
            k = k+1
            print(k)
        except:
            print('DOWNLOAD ERROR', audio_filename)
            continue           
print('Audios of presences OK')

Audios of presences OK


In [14]:
### absences ###
q = 0
for i in list(set(absences.url)): # loop over recordings
    tmp = absences[absences.url==i]
    audio_filename = tmp.iloc[0].url.split('download/')[1]
    if not os.path.exists(recording_dir+audio_filename):
        try:
            urllib.request.urlretrieve(tmp.iloc[0].url, recording_dir+audio_filename)
            q = q+1
            print(q)
        except:
            print('DOWNLOAD ERROR', audio_filename)
            continue
print('Audios of absences OK')

Audios of absences OK


In [15]:
%%time
window_length = 2 # sample time-window length in seconds
missed = 0

for i in list(set(presences.url)): # loop over recordings
        
    tmp = presences[presences.url==i]
    audio_filename = tmp.iloc[0].url.split('download/')[1]
    
    for c in range(len(tmp)): # loop over labels in the same recording
        sound_start, sound_end = [tmp.iloc[c]['x1'], tmp.iloc[c]['x2']]
        specie = tmp.iloc[c].species
        #callshape = tmp.iloc[c].callshape
        index_tmp = tmp.iloc[c].index
        #print(tmp.iloc[c].id)
            
        if not os.path.exists(dataset_dir_p+str(specie)):
            os.mkdir(dataset_dir_p+str(specie))

        shft = ((sound_end-sound_start)-window_length)/2
        start_sample = round(sampling_rate*(sound_start+shft))
        end_sample = start_sample+round(sampling_rate*window_length)
        #wav_name = tmp.iloc[c].recording.split('.')[0]
        #arbimon_minute = tmp.iloc[c].recording.split('.')[1]
        #minute_number = int(arbimon_minute.split('p')[1]) - 1 
        #site = tmp.iloc[c].site
        #filename_info = site+'-'+audio_filename+'-'+wav_name+'-'+'min'+str(minute_number)+'sec'+str(round(start_sample/sampling_rate,2))
        filename = 'id_'+str(tmp.iloc[c].id)+'.png'
        #print(filename)
            
        if not os.path.exists(dataset_dir_p+str(specie)+'/'+filename):
            try:
                audio_data, sampling_rate = librosa.load(recording_dir+audio_filename, sr=sampling_rate)
            except Exception as e:
                missed = missed + 1
                print(str(missed),'ERROR: LIBROSA LOAD\n', filename)
                continue
            
            try:
                S = librosa.feature.melspectrogram(y = audio_data[int(start_sample):int(end_sample)], 
                                                   sr = sampling_rate,
                                                   n_fft = 2048,
                                                   n_mels = 128,
                                                   hop_length = 512, 
                                                   win_length = 1024)
                dpi=100
                fig = plt.figure(num=None, figsize=(300/dpi, 300/dpi), dpi=dpi)
                #plt.subplot(222)
                ax = plt.axes()
                ax.set_axis_off()
                librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
                plt.savefig(dataset_dir_p+str(specie)+'/'+filename, bbox_inches='tight', transparent=True, pad_inches=0.0)
                plt.close()
            except Exception as e:
                missed = missed + 1
                print(str(missed),'ERROR LIBROSA FEATURE ### id: ', tmp.iloc[c].id, '### ', 
                      'start sample: ' ,start_sample)
                continue

print('Spectrograms OK\n', 'Total missed samples: ', missed)

Spectrograms OK
 Total missed samples:  0
CPU times: user 338 ms, sys: 12 ms, total: 350 ms
Wall time: 360 ms


In [16]:
%%time
# Repeat previous cell for absences

window_length = 2 # sample time-window length in seconds
missed = 0

for i in list(set(absences.url)): # loop over recordings
        
    tmp = absences[absences.url==i]
    audio_filename = tmp.iloc[0].url.split('download/')[1]
    
    for c in range(len(tmp)): # loop over labels in the same recording
        sound_start, sound_end = [tmp.iloc[c]['x1'], tmp.iloc[c]['x2']]
        specie = tmp.iloc[c].species
        #callshape = tmp.iloc[c].callshape
        index_tmp = tmp.iloc[c].index
            
        if not os.path.exists(dataset_dir_a+str(specie)):
            os.mkdir(dataset_dir_a+str(specie))

        shft = ((sound_end-sound_start)-window_length)/2
        start_sample = round(sampling_rate*(sound_start+shft))
        end_sample = start_sample+round(sampling_rate*window_length)
        filename = 'id_'+str(tmp.iloc[c].id)+'.png'
            
        if not os.path.exists(dataset_dir_a+str(specie)+'/'+filename):
            try:
                audio_data, sampling_rate = librosa.load(recording_dir+audio_filename, sr=sampling_rate)
            except Exception as e:
                missed = missed + 1
                print(str(missed),'ERROR: LIBROSA LOAD\n', filename)
                continue
            
            try:
                S = librosa.feature.melspectrogram(y = audio_data[int(start_sample):int(end_sample)], 
                                                   sr = sampling_rate,
                                                   n_fft = 2048,
                                                   n_mels = 128,
                                                   hop_length = 512, 
                                                   win_length = 1024)
                dpi=100
                fig = plt.figure(num=None, figsize=(300/dpi, 300/dpi), dpi=dpi)
                #plt.subplot(222)
                ax = plt.axes()
                ax.set_axis_off()
                librosa.display.specshow(librosa.power_to_db(S, ref=np.max))
                plt.savefig(dataset_dir_a+str(specie)+'/'+filename, bbox_inches='tight', transparent=True, pad_inches=0.0)
                plt.close()
            except Exception as e:
                missed = missed + 1
                print(str(missed),'ERROR LIBROSA FEATURE ### id: ', tmp.iloc[c].id, '### ', 
                      'start sample: ' ,start_sample)
                continue

print('Spectrograms OK\n', 'Total missed samples: ', missed)

/home/gabsoni/miniconda3/envs/exp1/lib/python3.9/site-packages/librosa/core/spectrum.py:222: UserWarning: n_fft=2048 is too small for input signal of length=0
  warnings.warn(


1 ERROR LIBROSA FEATURE ### id:  227332676 ###  start sample:  -196
2 ERROR LIBROSA FEATURE ### id:  227375350 ###  start sample:  -452
3 ERROR LIBROSA FEATURE ### id:  227361579 ###  start sample:  -452
4 ERROR LIBROSA FEATURE ### id:  227267373 ###  start sample:  -68
5 ERROR LIBROSA FEATURE ### id:  227383536 ###  start sample:  -708
6 ERROR LIBROSA FEATURE ### id:  227262126 ###  start sample:  -68
7 ERROR LIBROSA FEATURE ### id:  227383897 ###  start sample:  -708
8 ERROR LIBROSA FEATURE ### id:  227400881 ###  start sample:  -708
9 ERROR LIBROSA FEATURE ### id:  227359829 ###  start sample:  -708
10 ERROR LIBROSA FEATURE ### id:  227399497 ###  start sample:  -452
Spectrograms OK
 Total missed samples:  10
CPU times: user 19min 39s, sys: 25.3 s, total: 20min 4s
Wall time: 5min 7s
